In [1]:
# Download pops github repo
!git clone https://github.com/FinucaneLab/pops.git

# Copy pops.ipynb to pops folder
!cp pops.ipynb pops/

# Change directory to pops and make sure we are now in "TWAS/Pops/pops"
%cd pops/

Cloning into 'pops'...
remote: Enumerating objects: 235, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 235 (delta 26), reused 23 (delta 12), pack-reused 196
Receiving objects: 100% (235/235), 108.46 MiB | 15.10 MiB/s, done.
Resolving deltas: 100% (103/103), done.
Updating files: 100% (25/25), done.
/Users/JerryYaw/Documents/TWAS/Pops/pops
/Users/JerryYaw/Documents/TWAS/Pops/pops


# Environment Setup
Please download the **pops pipeline data**, **magma software** and **gene features** to the current pops folder.<br>
And arrange the feature folders according to the Tips below<br>

<h3>Download PoPS Pipeline Data</h3>
<pre><code>Please choose the gene prioritization results to download and save as data in pops.
<a href="https://www.finucanelab.org/data">Finucane Lab Data</a>
</code></pre>

<h3>Download Magma software</h3>
<pre><code>Please find the compiler and operation system that suits you, and unzip to the pops folder!
<a href="https://cncr.nl/research/magma/">CNCR CTG Lab Magma</a>
</code></pre>

<h3>Download Gene Features</h3>
<pre><code>We use human_lung and human_pbmc for COVID19 pops pipeline 
<a href="https://github.com/FinucaneLab/gene_features/tree/master/features">Finucane Lab Features</a>
</code></pre>

<div style="background-color: #ADD8E6; color: black; border-left: 5px solid black; padding: 10px;">
    <b>Tip:</b> Please refer to the pops example's data folder and arrange human_lung and human_pbmc similarly. This means you can create human_lung and humna_pmbc folders under the data folder and copy four folders (features_munged,features_raw,magma_scores, and utils) from the example data for each, replacing the files in features_raw with the download features(keep GTEx.txt). 
</div>

In [2]:
import os
import requests
import gzip
import shutil
from concurrent.futures import ThreadPoolExecutor

# Download and preprocessing COVID19 GWAS Summary data
download_dir = "data_covid"
os.makedirs(download_dir, exist_ok=True)

urls = [
    "https://storage.googleapis.com/covid19-hg-public/20200915/results/20201020/COVID19_HGI_A1_ALL_20201020.txt.gz",
    "https://storage.googleapis.com/covid19-hg-public/20200915/results/20201020/COVID19_HGI_A2_ALL_leave_23andme_20201020.txt.gz",
    "https://storage.googleapis.com/covid19-hg-public/20200915/results/20201020/COVID19_HGI_B1_ALL_20201020.txt.gz",
    "https://storage.googleapis.com/covid19-hg-public/20200915/results/20201020/COVID19_HGI_B2_ALL_leave_23andme_20201020.txt.gz",
    "https://storage.googleapis.com/covid19-hg-public/20200915/results/20201020/COVID19_HGI_C1_ALL_leave_23andme_20201020.txt.gz",
    "https://storage.googleapis.com/covid19-hg-public/20200915/results/20201020/COVID19_HGI_C2_ALL_leave_23andme_20201020.txt.gz",
    "https://storage.googleapis.com/covid19-hg-public/20200915/results/20201020/COVID19_HGI_D1_ALL_20201020.txt.gz",
]

def download_and_unzip(url):
    local_filename = url.split('/')[-1]
    dest_path = os.path.join(download_dir, local_filename)
    txt_filename = local_filename.rstrip('.gz')
    txt_path = os.path.join(download_dir, txt_filename)

    if not os.path.exists(dest_path):
        print(f"Downloading {local_filename}...")
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Check for errors
        with open(dest_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

    print(f"Unzipping {local_filename}...")
    with gzip.open(dest_path, 'rb') as f_in:
        with open(txt_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    os.remove(dest_path)  # Remove all .gz files
    print(f"Completed {local_filename}.")

# Download and unzip files in parallel
with ThreadPoolExecutor(max_workers=4) as executor:
    executor.map(download_and_unzip, urls)

# Merge all COVID19 GWAS summary files into one
summary_stats_path = os.path.join(download_dir, "COVID19_GWAS_summary_stats.txt")

# Sort the files Before merging
txt_files = [f for f in os.listdir(download_dir) if f.endswith(".txt")]
txt_files.sort()

with open(summary_stats_path, 'wb') as outfile:
    for i, filename in enumerate(txt_files):
        file_path = os.path.join(download_dir, filename)
        with open(file_path, 'rb') as readfile:
            # Skip the header for all but the first file
            if i != 0:
                next(readfile)  # Skip the first line (header) of subsequent files
            shutil.copyfileobj(readfile, outfile)

# Remove all .txt files 
for filename in txt_files:
    if filename != os.path.basename(summary_stats_path):
        os.remove(os.path.join(download_dir, filename))

print("All files downloaded, unzipped, and combined.")

Unzipping COVID19_HGI_B2_ALL_leave_23andme_20201020.txt.gz...
Unzipping COVID19_HGI_A1_ALL_20201020.txt.gz...
Unzipping COVID19_HGI_A2_ALL_leave_23andme_20201020.txt.gz...
Completed COVID19_HGI_A1_ALL_20201020.txt.gz.
Completed COVID19_HGI_B2_ALL_leave_23andme_20201020.txt.gz.
Unzipping COVID19_HGI_B1_ALL_20201020.txt.gz...
Completed COVID19_HGI_A2_ALL_leave_23andme_20201020.txt.gz.
Unzipping COVID19_HGI_C1_ALL_leave_23andme_20201020.txt.gz...
Completed COVID19_HGI_B1_ALL_20201020.txt.gz.
Unzipping COVID19_HGI_D1_ALL_20201020.txt.gz...
Unzipping COVID19_HGI_C2_ALL_leave_23andme_20201020.txt.gz...
Completed COVID19_HGI_C1_ALL_leave_23andme_20201020.txt.gz.
Completed COVID19_HGI_D1_ALL_20201020.txt.gz.
Completed COVID19_HGI_C2_ALL_leave_23andme_20201020.txt.gz.
All files downloaded, unzipped, and combined.


# hg38 reference panel (optional)
The PoPS Pipeline Data includes a reference panel of 1000G.EUR European genome data. If you want to use hg38 instead, please download the Plink 2.0 software and move its executable file "plink2.0" to the pops folder so the following script can run.
<h3>Download Plink</h3>
Please choose the operation system and version that suits you, and unzip it to the pops folder!
<pre><code>
<a href="https://www.cog-genomics.org/plink/2.0/">Christopher Chang Plink</a>
</code></pre>

<div style="background-color: pink; color: black; border-left: 5px solid black; padding: 10px;">
    <b>Tip:</b> The common system path in Mac is "/usr/local/bin/" or "~/bin/" After placing the executed file, you can use plink or plink --version to check if the installation succeeded.
</div>

In [1]:
# Create hg38 reference panel(optional)
!cp ../getRefHg38.sh .
!yes | bash getRefHg38.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    62    0    62    0     0    170      0 --:--:-- --:--:-- --:--:--   170
100   320  100   320    0     0    380      0 --:--:-- --:--:-- --:--:--     0
100 3239M  100 3239M    0     0  64.9M      0  0:00:49  0:00:49 --:--:-- 69.4M   0  0:00:59  0:00:08  0:00:51 68.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   240    0   240    0     0    369      0 --:--:-- --:--:-- --:--:--   369
100   475    0   475    0     0    398      0 --:--:--  0:00:01 --:--:--   398
100 2748M  100 2748M    0     0  46.4M      0  0:00:59  0:00:59 --:--:-- 45.9M5.2M      0  0:01:17  0:00:07  0:01:10 47.7M0  46.4M      0  0:00:59  0:00:58  0:00:01 46.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
 

# Add codes to solve a common problem before running 
The "ValueError: shapes (18383,743) and (1867) not aligned: 743 (dim 1) != 1867 (dim 0)" issue is caused by the duplicated name in the generated matrix table. A **solution provided by Vinodsri** in the GitHub issue is adding codes renaming the feature clusters in the **munge_feature_directory.py** script.

<h3>Insert the code chunk in the position and import os</h3>
<pre><code>
import os
    
for f in all_feature_files:
    f_df = pd.read_csv(f, sep="\t", index_col=0).astype(np.float64)
    f_df = gene_annot_df.merge(
        f_df, how="left", left_index=True, right_index=True)
    <span style="color: brown;"># Add the following three line codes accordingly to munge_feature_directory.py</span>
    base_filename = os.path.basename(f)
    fname = base_filename.replace(r".txt.gz", "_")
    f_df.columns = f_df.columns.str.replace(r"Cluster", fname)
    <span style="color: brown;"># -----------------------------  by vinodhsri  ------------------------------</span>
    if nan_policy == "raise":
        assert not f_df.isnull().values.any(), "Missing genes in feature matrix."
</code></pre>

<h3>Now, that's start the pops pipelines with hg38 reference panel! </h3>

In [9]:
# Calling Pops0 step 
!cp ../runPopsi0.sh .
!bash runPopsi0.sh

Process feature files to feature matrix file...
Feature matrix process completed


In [7]:
# Calling Pops1 step
!cp ../runPopsi1.sh .
!bash runPopsi1.sh

Calculate magma score for each GWAS summary statistic file...
Welcome to MAGMA v1.10 (custom)
Using flags:
	--bfile data/all_hg38
	--gene-annot data/magma_0kb.genes.annot
	--pval data_covid/COVID19_GWAS_summary_stats.txt use=13,9 ncol=11
	--gene-model snp-wise=mean
	--out data/human_lung/magma_scores/lung-COVID19

Start time is 10:18:00, Monday 08 Apr 2024

Loading PLINK-format data...
Reading file data/all_hg38.fam... 3202 individuals read
Reading file data/all_hg38.bim... 70883903 SNPs read
Preparing file data/all_hg38.bed... 

Reading SNP p-values from file data_covid/COVID19_GWAS_summary_stats.txt... 
	detected 13 variables in file
	using variable: variable 13 (SNP id)
	using variable: variable 9 (p-value)
	using variable: variable 11 (sample size; discarding SNPs with N < 50)
	read 84817347 lines from file, containing valid SNP p-values for 2588143 SNPs in data (3.051% of lines, 3.454% of SNPs in data)
	         dropped all occurrences of each from analysis
	         writing list 

In [10]:
# Calling Pops2 step
!cp ../runPopsi2.sh .
!bash runPopsi2.sh

Run Pops for COVID19 GWAS summary statistic file...
Pops COVID19 pipeline completed
